# Yandex Algorithm 2018 ML Track

## простой baseline на основе Facebook fastText 
при подкрутке параметров может давать более 84000-85500

Будем использовать оригинальный python binding от Facebook:
  * основная страница https://fasttext.cc/docs/en/english-vectors.html
  * python binding собирао из trunk: https://github.com/facebookresearch/fastText/tree/master/python
  * обученную модель брал со следующей страницы: https://fasttext.cc/docs/en/pretrained-vectors.html

Весь workflow скопирван с предыдущего baseline от Cheremisin Oleg:
    https://github.com/applied-data-science/Data_Mining_in_Action_2018_Spring/blob/master/sport/hw2_yandex_algorithm_2018/benchmarks/Cheremisin_Oleg_easy_tfidf_baseline.ipynb

In [1]:
import fastText

In [2]:
ft_model = fastText.load_model("./fastText/cc.ru.300.bin/data")

In [3]:
import pandas as pd
import numpy as np

In [4]:
train = pd.read_csv('datasets/yandex/train.tsv', sep='\t', quoting=3, header=None, encoding="utf-8")

In [5]:
test = pd.read_csv('datasets/yandex/public.tsv', sep='\t', quoting=3, error_bad_lines=False, header=None, encoding="utf-8")

In [6]:
test.shape

(9968, 6)

In [7]:
train.fillna('', inplace=True)
test.fillna('', inplace=True)

In [8]:
train.sample(3)

,0,1,2,3,4,5,6,7
2019,5685407011325,,,это было ярко .,4,что же это было ?,neutral,0.543724
14773,42739915871465,,,извините меня .,2,ты виновата и ты извиняешься ?,good,0.960363
77388,223388035859475,,нам нужно идти .,"да , нужно .",1,"сейчас у меня занятия , но я могла бы их отмен...",good,0.974046


In [9]:
test.sample(3)

,0,1,2,3,4,5
544,14337737683950,у меня не много общего с парнями из братства .,я из братства .,ты же заинтересован во мне .,5,"значит , во мне ."
3070,87613720927558,,,садись .,0,"не , я постою ."
972,28240451241798,,райан зовет .,я должна идти .,0,ты иди .


In [10]:
# расставляем значения по формуле хорошесть * confidence
# Good (2)
# Neutral (1)
# Bad (0)

In [11]:
def rank2num(st):
    if st == 'good':
        return 2
    else:
        if st == 'neutral':
            return 1
        else:
            return 0

In [12]:
train['rank'] = train[6].apply(rank2num)
train['target'] = train['rank'] * train[7]
train.head()

,0,1,2,3,4,5,6,7,rank,target
0,22579918886,"кликни на меня а потом на надпись "" видео - зв...","о , я тебя вижу .","ладно , повесь трубку .",0,не могу .,good,0.875352,2,1.750703
1,22579918886,"кликни на меня а потом на надпись "" видео - зв...","о , я тебя вижу .","ладно , повесь трубку .",1,"нет , звонить буду я .",neutral,0.900968,1,0.900968
2,22579918886,"кликни на меня а потом на надпись "" видео - зв...","о , я тебя вижу .","ладно , повесь трубку .",2,"слушай , я не мог уйти .",bad,0.884320,0,0.000000
3,22579918886,"кликни на меня а потом на надпись "" видео - зв...","о , я тебя вижу .","ладно , повесь трубку .",3,я не прекращу звонить .,good,0.982530,2,1.965061
4,22579918886,"кликни на меня а потом на надпись "" видео - зв...","о , я тебя вижу .","ладно , повесь трубку .",4,я звоню им .,good,0.838054,2,1.676107


In [13]:
t1_ft = np.vstack(train[1].apply(lambda x: ft_model.get_sentence_vector(x)))
t2_ft = np.vstack(train[2].apply(lambda x: ft_model.get_sentence_vector(x)))
t3_ft = np.vstack(train[3].apply(lambda x: ft_model.get_sentence_vector(x)))
t5_ft = np.vstack(train[5].apply(lambda x: ft_model.get_sentence_vector(x)))

te1_ft = np.vstack(test[1].apply(lambda x: ft_model.get_sentence_vector(x)))
te2_ft = np.vstack(test[2].apply(lambda x: ft_model.get_sentence_vector(x)))
te3_ft = np.vstack(test[3].apply(lambda x: ft_model.get_sentence_vector(x)))
te5_ft = np.vstack(test[5].apply(lambda x: ft_model.get_sentence_vector(x)))

In [14]:
%%time

X_train = np.hstack([t1_ft, t2_ft, t3_ft, t5_ft])
X_test  = np.hstack([te1_ft, te2_ft, te3_ft, te5_ft])

CPU times: user 87.1 ms, sys: 81 ms, total: 168 ms
Wall time: 168 ms


In [15]:
X_train.shape, X_test.shape

((97533, 1200), (9968, 1200))

In [16]:
y_train = train['target']

In [17]:
train_part_size = int(0.75 * train['target'].shape[0])
X_train_part = X_train[:train_part_size, :]
y_train_part = y_train[:train_part_size]
X_valid =  X_train[train_part_size:, :]
y_valid = y_train[train_part_size:]

In [18]:
X_train_part.shape

(73149, 1200)

In [19]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso

reg = Ridge()

reg.fit(X_train_part, y_train_part)

Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)

In [20]:
reg_pred = reg.predict(X_valid)

In [21]:
# Oleg говорит, что метрика ниже - неправильная, но на практике её снижение даёт уверенный рост на лидерборде

In [22]:
%%time
from sklearn.metrics import mean_absolute_error

valid_mae = mean_absolute_error(y_valid, reg_pred)
print(valid_mae)

0.7190278713445925
CPU times: user 1.03 ms, sys: 302 µs, total: 1.33 ms
Wall time: 796 µs


In [23]:
%%time
reg.fit(X_train, y_train)

CPU times: user 10.6 s, sys: 1.47 s, total: 12.1 s
Wall time: 1.42 s


Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)

In [24]:
y_test = reg.predict(X_test)

In [25]:
sub = pd.DataFrame()

In [26]:
test.sample(3)

,0,1,2,3,4,5
7518,210556936433923,даже не надейся .,бун чудом избежал смерти .,"убил григгса , своего спасителя .",1,лесси убил уильямса ?
6168,175687008644796,я за ключами от машины схожу .,хорошо ?,ага .,4,"возьми ключи , и можешь ехать ."
665,18707487932940,"' потому , что ты занимался этим с цветком ?","нет , нет , нет .","по тому , что я был одержим работой .",0,я не повернусь назад .


In [27]:
sub['context_id'] = test[0]
sub['reply_id'] = test[4]
sub['rank'] = - y_test

In [28]:
sub.head()

,context_id,reply_id,rank
0,138920940977,0,-1.034709
1,138920940977,1,-1.335503
2,138920940977,2,-1.060807
3,138920940977,3,-1.462563
4,138920940977,4,-1.494854


In [29]:
submission = sub.sort_values(by=['context_id', 'rank'])

In [30]:
del submission['rank']

In [31]:
submission.head()

,context_id,reply_id
4,138920940977,4
3,138920940977,3
1,138920940977,1
5,138920940977,5
2,138920940977,2


In [32]:
test.shape, sub.shape

((9968, 6), (9968, 3))

In [34]:
submission.to_csv('yandex-ml-naive-fasttext.tsv',header=None, index=False, sep=' ')